In [78]:
import pandas as pd
from datetime import timedelta, timezone

In [79]:
df = pd.read_excel('events1.xlsx')
df = df[['Name', 'Ticker', 'Date', 'Event Type', 'Time', 'Period']]
def clean_period(period):
    if period is None or pd.isna(period):
        return period
    else:
        period = str(period)
        if '(' in period:
            split_period = period.split('(')[0].strip()
            return split_period
        else:
            return period
def clean_time(t):
    if pd.isna(t):
        return t
    t = str(t).replace(':', '').zfill(4)
    return t
df['Period'] = df['Period'].apply(clean_period)
df['Time'] = df['Time'].apply(clean_time)
df = df.drop_duplicates()
df['Date'] = df['Date'].dt.strftime('%Y%m%d')
df

,Name,Ticker,Date,Event Type,Time,Period
0,Nextage Co Ltd,3186 JP,20260105,ER,NaN,Y 25
2,ASKUL Corp,2678 JP,20260105,SR,NaN,25
3,Hisamitsu Pharmaceutical Co In,4530 JP,20260106,ER,1430,Q3 26
4,Hisamitsu Pharmaceutical Co In,4530 JP,20260106,EC,NaN,Q3 26
5,Takashimaya Co Ltd,8233 JP,20260106,ER,NaN,Q3 26
...,...,...,...,...,...,...
836,Northsand Inc,446A JP,20260322,ER,NaN,Y 26
837,FUNDINNO Inc,462A JP,20260323,ER,NaN,Q1 26
838,BRANU Inc,460A JP,20260323,ER,NaN,Q1 26
839,HUMAN MADE Inc,456A JP,20260323,ER,NaN,Y 26


In [80]:
HKT = timezone(timedelta(hours=8))      # Hong Kong time (UTC+8, no DST)
DEFAULT_DURATION_MIN = 60              # adjust if you prefer 30, 15, etc.

def escape_ics_text(s: str) -> str:
    """Escape text per RFC 5545 for DESCRIPTION/SUMMARY fields."""
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s = str(s)
    s = s.replace("\\", "\\\\")
    s = s.replace(";", r"\;")
    s = s.replace(",", r"\,")
    s = s.replace("\r\n", "\n").replace("\r", "\n").replace("\n", r"\n")
    return s

def fold_ics_line(line: str, limit: int = 75) -> str:
    """Fold lines at 75 octets (approx). Good enough for ASCII-heavy text."""
    # For most tickers/names this is sufficient; if you have lots of non-ascii,
    # folding-by-bytes would be more exact.
    if len(line) <= limit:
        return line
    out = []
    while len(line) > limit:
        out.append(line[:limit])
        line = " " + line[limit:]
    out.append(line)
    return "\r\n".join(out)

def build_uid(ticker: str, dt_utc_str: str) -> str:
    # stable-ish UID; if you later want updates not duplicates, keep UID stable per event
    t = "".join(str(ticker).split())  # remove spaces
    return f"{t}-{dt_utc_str}@earnings.ivancheung"

def parse_row_dt_fields(row) -> dict[str, str]:
    """
    Create DTSTART/DTEND in UTC Zulu format.
    Returns (dtstart_utc, dtend_utc, dtstamp_utc) as strings: YYYYMMDDTHHMMSSZ
    """
    dtstamp = pd.Timestamp.utcnow().strftime("%Y%m%dT%H%M%SZ")
    date_str = str(row["Date"])  # expected YYYYMMDD already in your pipeline
    time_val = row.get("Time", None)

    if pd.isna(time_val):
        # All-day event
        d = pd.to_datetime(date_str, format="%Y%m%d", errors="coerce")
        if pd.isna(d):
            raise ValueError(f"Invalid Date '{row['Date']}' for all-day event.")
        start_date = d.strftime("%Y%m%d")
        end_date = (d + pd.Timedelta(days=1)).strftime("%Y%m%d")
        return {
            "DTSTAMP": dtstamp,
            "DTSTART_LINE": f"DTSTART;VALUE=DATE:{start_date}",
            "DTEND_LINE": f"DTEND;VALUE=DATE:{end_date}",
            "IS_ALL_DAY": True,
        }
    else:
        # Timed event (interpret as HKT local then convert to UTC)
        time_str = str(time_val).strip()
        local_dt = pd.to_datetime(date_str + time_str, format="%Y%m%d%H%M", errors="coerce")
        if pd.isna(local_dt):
            raise ValueError(f"Invalid Date/Time '{row['Date']} {time_val}' for timed event.")
        local_dt = local_dt.to_pydatetime().replace(tzinfo=HKT)
        utc_dt = local_dt.astimezone(timezone.utc)
        utc_end = utc_dt + timedelta(minutes=DEFAULT_DURATION_MIN)
        dtstart = utc_dt.strftime("%Y%m%dT%H%M%SZ")
        dtend = utc_end.strftime("%Y%m%dT%H%M%SZ")
        return {
            "DTSTAMP": dtstamp,
            "DTSTART_LINE": f"DTSTART:{dtstart}",
            "DTEND_LINE": f"DTEND:{dtend}",
            "IS_ALL_DAY": False,
        }

def make_summary(row) -> str:
    name = escape_ics_text(row.get("Name", ""))
    ticker = escape_ics_text(row.get("Ticker", ""))
    period = escape_ics_text(row.get("Period", ""))
    evtype = escape_ics_text(row.get("Event Type", ""))

    # Keep summary compact and consistent
    base = f"{ticker[:4]} {name}"
    if period:
        if evtype:
            if evtype == 'SR':
                return f"{base} {period} Sales Result"
            if evtype == 'ER':
                return f"{base} {period} Earnings Release"
            if evtype == 'EC':
                return f"{base} {period} Earnings Call"
    return f"{base} Earnings"

def make_description(row) -> str:
    # Keep this minimal; add fields only if you truly want them in the calendar body
    return ""

# Build ICS
lines = [
    "BEGIN:VCALENDAR",
    "VERSION:2.0",
    "PRODID:-//EarningsCalendar//ExcelToICS//EN",
    "CALSCALE:GREGORIAN",
]

for _, row in df.iterrows():
    dt_fields = parse_row_dt_fields(row)
    summary = make_summary(row)
    description = make_description(row)

    uid = build_uid(row.get("Ticker", ""), dt_fields["DTSTART_LINE"])

    event_lines = [
        "BEGIN:VEVENT",
        f"UID:{uid}",
        f"DTSTAMP:{dt_fields['DTSTAMP']}",
        dt_fields["DTSTART_LINE"],
        dt_fields["DTEND_LINE"],
        f"SUMMARY:{summary}",
        "CATEGORIES:quarter_earning",
    ]
    if description:
        event_lines.append(f"DESCRIPTION:{description}")

    event_lines.append("END:VEVENT")

    # Fold lines for ICS compliance
    for el in event_lines:
        lines.append(fold_ics_line(el))

lines.append("END:VCALENDAR")

ics_text = "\r\n".join(lines) + "\r\n"

# Write file
with open("earnings.ics", "w", encoding="utf-8", newline="") as f:
    f.write(ics_text)

print("Wrote earnings.ics with", len(df), "events")


Wrote earnings.ics with 729 events


In [81]:
import calendar


In [82]:
def monthly_df_to_ics(
    df_monthly: pd.DataFrame,
    output_path: str = "monthly_data.ics",
    prodid: str = "-//MonthlyDataCalendar//ExcelToICS//EN",
) -> str:
    """
    Convert an analyst-input monthly data table into an ICS file (awareness calendar).

    Expected columns (minimum):
      - Active (Y/N)
      - Item_Type (EQUITY or MACRO)
      - Country (e.g., JPN, KR)
      - Ticker (nullable; equities)
      - Name (nullable; equities)
      - Metric (nullable; equities)
      - Topic (nullable; macro)
      - Rule_Raw (exact rule string, JP/EN)
      - Rule_Class (DOM_WINDOW, DOM_RANGE, DOM_AFTER, MONTH_START, MONTH_END, NTH_WEEKDAY, BIZ_APPROX)
      - Start_Day (int; for DOM_*, MONTH_*, BIZ_APPROX; can be blank for NTH_WEEKDAY)
      - End_Day (int; for DOM_RANGE/DOM_AFTER/MONTH_* / BIZ_APPROX; can be blank for NTH_WEEKDAY)
      - Nth (int; for NTH_WEEKDAY)
      - Weekday (MO,TU,WE,TH,FR,SA,SU; for NTH_WEEKDAY)
      - Publish_Method (optional)
      - Released_By (optional)
      - Website (optional)
      - Note (optional)

    Output:
      - All-day events
      - Monthly recurrence (RRULE)
      - CATEGORIES:monthly_data
      - TRANSP:TRANSPARENT
      - Minimal but robust VEVENT fields: UID, DTSTAMP, DTSTART, DTEND, SUMMARY, DESCRIPTION, RRULE
    """

    # ---------- helpers ----------
    def _is_active(v) -> bool:
        return str(v).strip().upper() in {"Y", "YES", "TRUE", "1"}

    def _safe(v) -> str:
        if v is None or (isinstance(v, float) and pd.isna(v)):
            return ""
        return str(v).strip()

    def escape_ics_text_local(s: str) -> str:
        # Use your existing escape_ics_text if you fix it; for now keep safe escaping here.
        if s is None:
            return ""
        s = str(s)
        s = s.replace("\\", "\\\\")
        s = s.replace(";", r"\;")
        s = s.replace(",", r"\,")
        s = s.replace("\r\n", "\n").replace("\r", "\n").replace("\n", r"\n")
        return s

    def fold(line: str) -> str:
        return fold_ics_line(line)

    def weekday_to_ics(w: str) -> str:
        w = _safe(w).upper()
        allowed = {"MO", "TU", "WE", "TH", "FR", "SA", "SU"}
        if w not in allowed:
            raise ValueError(f"Invalid Weekday '{w}'. Use one of {sorted(allowed)}.")
        return w

    def build_uid_monthly(event_key: str) -> str:
        # Stable UID: do NOT include a date so recurring event remains one logical VEVENT.
        ek = "".join(_safe(event_key).split())
        if not ek:
            raise ValueError("Event_Key is required to build UID.")
        return f"{ek}@monthly.ivancheung"

    def build_summary(row: pd.Series) -> str:
        item_type = _safe(row.get("item_type", "")).upper()
        country = _safe(row.get("country", ""))
        ticker = str(_safe(row.get("ticker", "")))
        name = _safe(row.get("name", ""))
        metric = _safe(row.get("metric", ""))
        topic = _safe(row.get("topic", ""))

        if item_type == "EQUITY":
            # Per your rule: "6563 Toyota Monthly Sales"
            # If metric is empty, fall back to Name.
            base = " ".join([p for p in [ticker, name, metric] if p])
            return escape_ics_text_local(base) if base else escape_ics_text_local(ticker or name or "Equity Monthly Data")

        # MACRO
        # Per your rule: Summary can be Topic + Country, while description holds details.
        base = " ".join([p for p in [topic, country] if p])
        return escape_ics_text_local(base) if base else escape_ics_text_local(topic or f"Macro Monthly Data {country}")

    def build_description(row: pd.Series) -> str:
        # Equity: description shows exact release rule and source (if exists)
        # Macro: include topic line + exact rule + website/source if exists
        parts = []

        item_type = _safe(row.get("item_type", "")).upper()
        country = _safe(row.get("country", ""))
        topic = _safe(row.get("topic", ""))

        rule_raw = _safe(row.get("rule_raw", ""))
        publish_method = _safe(row.get("publish_method", ""))
        released_by = _safe(row.get("released_by", ""))
        website = _safe(row.get("website", ""))
        note = _safe(row.get("note", ""))

        if rule_raw:
            parts.append(f"Rule: {rule_raw}")
        if publish_method:
            parts.append(f"Method: {publish_method}")
        if released_by:
            parts.append(f"Released by: {released_by}")
        if website:
            parts.append(f"Source: {website}")
        if note:
            parts.append(f"Note: {note}")

        return escape_ics_text_local("\n".join(parts))

    def build_rrule(row: pd.Series) -> str:
        rule_class = _safe(row.get("rule_class", "")).upper()

        if rule_class == "NTH_WEEKDAY":
            nth = row.get("nth", None)
            if nth is None or (isinstance(nth, float) and pd.isna(nth)):
                raise ValueError("Nth is required for NTH_WEEKDAY.")
            nth = int(nth)
            wd = weekday_to_ics(row.get("weekday", ""))
            return f"RRULE:FREQ=MONTHLY;BYDAY={wd};BYSETPOS={nth}"

        # Default: monthly recurrence by anchor day-of-month (DTSTART day drives it),
        # but we also set BYMONTHDAY explicitly for clarity.
        start_day = row.get("start_day", None)
        if start_day is None or (isinstance(start_day, float) and pd.isna(start_day)):
            raise ValueError(f"Start_Day is required for Rule_Class={rule_class}.")
        start_day = int(start_day)
        if not (1 <= start_day <= 31):
            raise ValueError(f"Start_Day must be 1..31, got {start_day}.")
        return f"RRULE:FREQ=MONTHLY;BYMONTHDAY={start_day}"

    def build_all_day_dt(row: pd.Series) -> tuple[str, str]:
        """
        Returns (DTSTART;VALUE=DATE:YYYYMMDD, DTEND;VALUE=DATE:YYYYMMDD)
        using a fixed reference month for DTSTART (2026-01) purely to define recurrence pattern.
        DTEND is exclusive.
        """
        rule_class = _safe(row.get("rule_class", "")).upper()

        # Choose a reference month; any month works for recurrence definition.
        ref_year, ref_month = 2026, 1

        if rule_class == "NTH_WEEKDAY":
            # Need a concrete DTSTART that matches the pattern in the reference month
            nth = int(row.get("nth"))
            wd = weekday_to_ics(row.get("weekday", ""))

            # Compute nth weekday in ref_year/ref_month
            first = pd.Timestamp(ref_year, ref_month, 1)
            # pandas weekday: Monday=0 ... Sunday=6
            wd_map = {"MO": 0, "TU": 1, "WE": 2, "TH": 3, "FR": 4, "SA": 5, "SU": 6}
            target = wd_map[wd]

            # Find first target weekday
            offset = (target - first.weekday()) % 7
            first_target = first + pd.Timedelta(days=offset)
            dt = first_target + pd.Timedelta(days=7 * (nth - 1))

            # All-day single day event: DTEND = next day
            dtstart = dt.strftime("%Y%m%d")
            dtend = (dt + pd.Timedelta(days=1)).strftime("%Y%m%d")
            return f"DTSTART;VALUE=DATE:{dtstart}", f"DTEND;VALUE=DATE:{dtend}"

        # For window/range/etc, use start_day/end_day directly in the reference month
        start_day = int(row.get("start_day"))
        end_day = row.get("end_day", None)

        # If End_Day missing, treat as single day
        if end_day is None or (isinstance(end_day, float) and pd.isna(end_day)):
            end_day = start_day
        end_day = int(end_day)

        # Clamp to 1..31 for awareness (MONTH_END uses 28..31); keep as entered.
        last_day = calendar.monthrange(ref_year, ref_month)[1]

        start_day_clamped = max(1, min(start_day, last_day))
        end_day_clamped   = max(1, min(end_day, last_day))

        start = pd.Timestamp(ref_year, ref_month, start_day_clamped)
        end   = pd.Timestamp(ref_year, ref_month, end_day_clamped)

        # Ensure start <= end
        if end < start:
            start, end = end, start

        # DTEND exclusive: add 1 day to end
        dtstart = start.strftime("%Y%m%d")
        dtend = (end + pd.Timedelta(days=1)).strftime("%Y%m%d")
        return f"DTSTART;VALUE=DATE:{dtstart}", f"DTEND;VALUE=DATE:{dtend}"

    df_use = df_monthly.copy()
    if "active" in df_use.columns:
        df_use = df_use[df_use["active"].apply(_is_active)]


    # ---------- build ICS ----------
    dtstamp_utc = pd.Timestamp.utcnow().strftime("%Y%m%dT%H%M%SZ")

    lines = [
        "BEGIN:VCALENDAR",
        "VERSION:2.0",
        f"PRODID:{prodid}",
        "CALSCALE:GREGORIAN",
    ]

    for _, row in df_use.iterrows():
        event_key = row.get("id", "")
        uid = build_uid_monthly(event_key=event_key)
        summary = build_summary(row)
        desc = build_description(row)
        rrule = build_rrule(row)
        dtstart_line, dtend_line = build_all_day_dt(row)

        event_lines = [
            "BEGIN:VEVENT",
            f"UID:{uid}",
            f"DTSTAMP:{dtstamp_utc}",
            dtstart_line,
            dtend_line,
            f"SUMMARY:{summary}",
            rrule,
            "CATEGORIES:monthly_data",
            "TRANSP:TRANSPARENT",
        ]
        if desc:
            event_lines.append(f"DESCRIPTION:{desc}")
        event_lines.append("END:VEVENT")

        for el in event_lines:
            lines.append(fold(el))

    lines.append("END:VCALENDAR")

    ics_text = "\r\n".join(lines) + "\r\n"

    with open(output_path, "w", encoding="utf-8", newline="") as f:
        f.write(ics_text)

    print(f"Wrote {output_path} with {len(df_use)} events (test mode: first 3 active rows).")
    return ics_text


In [83]:
df_monthly = pd.read_excel("monthly_release_rules_template.xlsx")
monthly_df_to_ics(df_monthly, output_path="monthly_data.ics")

Wrote monthly_data.ics with 71 events (test mode: first 3 active rows).


c:\Users\IvanCheung\OneDrive - Arrowpoint Investment Partners (Singapore) Pte. Ltd\Documents\arrowpoint\venv_data\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


"BEGIN:VCALENDAR\r\nVERSION:2.0\r\nPRODID:-//MonthlyDataCalendar//ExcelToICS//EN\r\nCALSCALE:GREGORIAN\r\nBEGIN:VEVENT\r\nUID:1@monthly.ivancheung\r\nDTSTAMP:20251229T052615Z\r\nDTSTART;VALUE=DATE:20260109\r\nDTEND;VALUE=DATE:20260112\r\nSUMMARY:7532 Pan Pacific International Ho Monthly Sales\r\nRRULE:FREQ=MONTHLY;BYMONTHDAY=9\r\nCATEGORIES:monthly_data\r\nTRANSP:TRANSPARENT\r\nDESCRIPTION:Rule: 10日頃\\nMethod: HP（メールでも配信）\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nUID:2@monthly.ivancheung\r\nDTSTAMP:20251229T052615Z\r\nDTSTART;VALUE=DATE:20260102\r\nDTEND;VALUE=DATE:20260105\r\nSUMMARY:2685 And St Hd Co Ltd Monthly Sales\r\nRRULE:FREQ=MONTHLY;BYMONTHDAY=2\r\nCATEGORIES:monthly_data\r\nTRANSP:TRANSPARENT\r\nDESCRIPTION:Rule: 第2営業日\\nMethod: 適時開示\\nNote: 決算月は決算時公表\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nUID:3@monthly.ivancheung\r\nDTSTAMP:20251229T052615Z\r\nDTSTART;VALUE=DATE:20260109\r\nDTEND;VALUE=DATE:20260112\r\nSUMMARY:Defense Monthly Order JP\r\nRRULE:FREQ=MONTHLY;BYMONTHDAY=9\r\nCATEGORIES:monthl